In [2]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import torchvision.transforms as transforms
import torch.nn.functional as F

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO("yolov8m.pt")  # Path to your YOLOv8 model
model.to('cpu') 

# Initialize video capture
video_path = r'C:\Users\ahmad\Downloads\853889-hd_1920_1080_25fps.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize variables
tracked_people = []  # List to hold tracked people (ID, x, y, last_frame)
person_id = 0  # Start with ID 0
frame_count = 0
max_distance = 50  # Max distance to consider matching people (can be tuned)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Detect people using YOLOv8
    results = model(frame)  # Results of detections
    boxes = results[0].boxes  # Get bounding boxes

    # Filter for "person" class (class ID 0)
    person_boxes = boxes[boxes.cls == 0]

    current_detections = []
    for box in person_boxes:
        # Extract bounding box and calculate the center
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        current_detections.append((center_x, center_y))

    # Track people manually by comparing current detections with previously tracked people
    new_tracked_people = []
    for (center_x, center_y) in current_detections:
        matched = False
        for person in tracked_people:
            prev_id, prev_x, prev_y, last_frame = person
            distance = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)
            if distance < max_distance:
                new_tracked_people.append((prev_id, center_x, center_y, frame_count))
                matched = True
                break
        
        if not matched:
            # Assign a new ID for this person
            new_tracked_people.append((person_id, center_x, center_y, frame_count))
            person_id += 1

    # Update the list of tracked people
    tracked_people = new_tracked_people

    # Draw the results on the frame
    for person in tracked_people:
        person_id, center_x, center_y, _ = person
        cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)
        cv2.putText(frame, f'ID: {person_id}', (center_x, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the output frame
    cv2.imshow('Tracked People', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 34 persons, 1 handbag, 1025.2ms
Speed: 2.0ms preprocess, 1025.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 1 handbag, 1032.2ms
Speed: 4.0ms preprocess, 1032.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 1 handbag, 1024.2ms
Speed: 2.0ms preprocess, 1024.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 persons, 1012.2ms
Speed: 3.0ms preprocess, 1012.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 1 backpack, 1030.2ms
Speed: 3.0ms preprocess, 1030.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 1015.2ms
Speed: 3.0ms preprocess, 1015.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 persons, 1014.2ms
Speed: 3.0ms preprocess, 1014.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 33 persons, 2 handb

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8m.pt")  # Path to your YOLOv8 model
model.to('cpu') 

# Initialize video capture
video_path = r'C:\Users\ahmad\Downloads\1338598-hd_1920_1080_30fps.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize variables
tracked_people = []  # List to hold tracked people (ID, x, y, last_frame)
person_id = 0  # Start with ID 0
frame_count = 0
max_distance = 50  # Max distance to consider matching people (can be tuned)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Detect people using YOLOv8
    results = model(frame)  # Results of detections
    boxes = results[0].boxes  # Get bounding boxes

    # Filter for "person" class (class ID 0)
    person_boxes = boxes[boxes.cls == 0]

    current_detections = []
    for box in person_boxes:
        # Extract bounding box coordinates, confidence, and center point
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        confidence = box.conf[0]  # Confidence score

        current_detections.append((center_x, center_y, x1, y1, x2, y2, confidence))

    # Track people manually by comparing current detections with previously tracked people
    new_tracked_people = []
    for (center_x, center_y, x1, y1, x2, y2, confidence) in current_detections:
        matched = False
        for person in tracked_people:
            prev_id, prev_x, prev_y, prev_x1, prev_y1, prev_x2, prev_y2, prev_conf, last_frame = person
            distance = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)
            if distance < max_distance:
                new_tracked_people.append((prev_id, center_x, center_y, x1, y1, x2, y2, confidence, frame_count))
                matched = True
                break

        
        if not matched:
            # Assign a new ID for this person
            new_tracked_people.append((person_id, center_x, center_y, x1, y1, x2, y2, confidence, frame_count))
            person_id += 1

    # Update the list of tracked people
    tracked_people = new_tracked_people

    # Draw the results on the frame
    for person in tracked_people:
        person_id, center_x, center_y, x1, y1, x2, y2, confidence, _ = person

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Display ID and confidence on top of the bounding box
        label = f'ID: {person_id} Conf: {confidence:.2f}'
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the output frame
    cv2.imshow('Tracked People', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 25 persons, 1 airplane, 2 backpacks, 5 handbags, 1041.2ms
Speed: 4.0ms preprocess, 1041.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 airplane, 1 backpack, 5 handbags, 999.2ms
Speed: 4.0ms preprocess, 999.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 airplane, 1 backpack, 3 handbags, 1001.2ms
Speed: 2.0ms preprocess, 1001.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 airplane, 1 backpack, 5 handbags, 999.2ms
Speed: 3.0ms preprocess, 999.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 persons, 1 airplane, 1 backpack, 3 handbags, 999.2ms
Speed: 3.0ms preprocess, 999.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 backpack, 6 handbags, 1015.2ms
Speed: 2.0ms preprocess, 1015.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8m.pt")  # Path to your YOLOv8 model
model.to('cpu') 

# Initialize video capture
video_path = r'C:\Users\ahmad\Downloads\1338598-hd_1920_1080_30fps.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize variables
tracked_people = []  # List to hold currently tracked people (ID, x, y, last_frame)
person_id = 0  # Start with ID 0
frame_count = 0
max_distance = 50  # Max distance to consider matching people (can be tuned)
history = []  # History of people who have left the frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Detect people using YOLOv8
    results = model(frame)  # Results of detections
    boxes = results[0].boxes  # Get bounding boxes

    # Filter for "person" class (class ID 0)
    person_boxes = boxes[boxes.cls == 0]

    current_detections = []
    for box in person_boxes:
        # Extract bounding box coordinates, confidence, and center point
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        confidence = box.conf[0]  # Confidence score

        current_detections.append((center_x, center_y, x1, y1, x2, y2, confidence))

    # Track people manually by comparing current detections with previously tracked people
    new_tracked_people = []
    for (center_x, center_y, x1, y1, x2, y2, confidence) in current_detections:
        matched = False
        # Check if this person matches a previously tracked person
        for person in tracked_people:
            prev_id, prev_x, prev_y, prev_x1, prev_y1, prev_x2, prev_y2, prev_conf, last_frame = person
            distance = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)
            if distance < max_distance:
                new_tracked_people.append((prev_id, center_x, center_y, x1, y1, x2, y2, confidence, frame_count))
                matched = True
                break

        # If no match is found, check the history for a potential returning person
        if not matched:
            for person in history:
                hist_id, hist_x, hist_y, last_seen_frame = person
                distance = np.sqrt((center_x - hist_x) ** 2 + (center_y - hist_y) ** 2)
                if distance < max_distance:
                    # Reassign the historical ID to this returning person
                    new_tracked_people.append((hist_id, center_x, center_y, x1, y1, x2, y2, confidence, frame_count))
                    history.remove(person)  # Remove from history since they're back in the frame
                    matched = True
                    break

        # If still no match, assign a new ID
        if not matched:
            new_tracked_people.append((person_id, center_x, center_y, x1, y1, x2, y2, confidence, frame_count))
            person_id += 1

    # Update the list of tracked people
    tracked_people = new_tracked_people

    # Move people who are no longer detected into the history
    for person in tracked_people:
        person_id, center_x, center_y, x1, y1, x2, y2, confidence, last_frame = person
        if frame_count - last_frame > 5:  # Adjust this threshold to tune sensitivity for disappearing people
            # If the person has been out of frame for some time, add them to the history
            history.append((person_id, center_x, center_y, last_frame))
            tracked_people.remove(person)

    # Draw the results on the frame
    for person in tracked_people:
        person_id, center_x, center_y, x1, y1, x2, y2, confidence, _ = person

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Display ID and confidence on top of the bounding box
        label = f'ID: {person_id} Conf: {confidence:.2f}'
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the output frame
    cv2.imshow('Tracked People', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 25 persons, 1 airplane, 2 backpacks, 5 handbags, 1091.2ms
Speed: 3.0ms preprocess, 1091.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 airplane, 1 backpack, 5 handbags, 1028.2ms
Speed: 3.0ms preprocess, 1028.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 airplane, 1 backpack, 3 handbags, 1017.2ms
Speed: 2.0ms preprocess, 1017.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 airplane, 1 backpack, 5 handbags, 1004.2ms
Speed: 2.0ms preprocess, 1004.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 persons, 1 airplane, 1 backpack, 3 handbags, 1009.2ms
Speed: 2.0ms preprocess, 1009.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 backpack, 6 handbags, 1003.2ms
Speed: 3.0ms preprocess, 1003.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: